In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

tmpfiles = '/content/tmp/'
train_tmp_folder = tmpfiles+'train_embeddinds/'
test_tmp_folder = tmpfiles+'test_embeddinds/'
val_tmp_folder = tmpfiles+'val_embeddinds/'

import os.path
from os import path

if path.exists(tmpfiles) == False:
  os.mkdir(tmpfiles)
else:
  if path.exists(train_tmp_folder) == False or path.exists(test_tmp_folder) == False or path.exists(val_tmp_folder) == False:
    try:
      os.mkdir(train_tmp_folder )
      os.mkdir(test_tmp_folder )
      os.mkdir(val_tmp_folder )
    except Exception:
      pass

!ls

#%cd /content/gdrive/MyDrive/ThermostabilityProject/
!pip install transformers tqdm
%cd /content/gdrive/MyDrive/ThermostabilityProject/


Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
datasets				 ref_models
drive-download-20230426T195016Z-001.zip  test_set_embeddings.pth
filename_pi.pth				 train_set_embeddings.pth
prepro_embeddings			 train_tmp_folder
ProtBERT_repro.ipynb			 val_set_embeddings.pth
README.md
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/content/gdrive/MyDrive/ThermostabilityProject


In [3]:
import pandas as pd
import numpy as np
import torch, torch.nn
from transformers import BertModel, BertTokenizer
import re, gc, os
import requests
from tqdm.auto import tqdm

In [4]:
def train_test_validation_splits(df):
    test,train_tot = df.loc[df['set']=='test'],df.loc[df['set']=='train']
    train, val = train_tot.loc[train_tot['validation']!=True], train_tot.loc[train_tot['validation']==True]
    return train,val, test

def embed_dataset(dataset_seqs, model, tokenizer, device, path, shift_left = 1, shift_right = -1):
  inputs_embedding = []

  cont=0
  for sample in tqdm(dataset_seqs):
    with torch.no_grad():
      ''' As we are using it in unsupervised manner, we are ignoring the CLS token and so, what's why 
          we are ignoring the special tokens (inspect better in huggingface why the three values in the output)'''
      #ids = tokenizer.batch_encode_plus(sample, add_special_tokens=True, padding=True, is_split_into_words=True, return_tensors="pt")      
      #ids = tokenizer.batch_encode_plus(sample, add_special_tokens=False, padding=True, is_split_into_words=True, return_tensors="pt")
      #embedding = model(input_ids=ids['input_ids'].to(device))[0]
      #inputs_embedding.append(embedding.detach().cpu().numpy())
      ids = tokenizer.encode_plus(list(sample.strip()), add_special_tokens=False, padding=True, is_split_into_words=True, return_tensors="pt")
      embedding = model(input_ids=ids['input_ids'].reshape(-1,1).to(device))[0]
      np.savez(path + str(cont)+'.npz',  embedding.detach().cpu().numpy() )
      cont+=1
      #inputs_embedding.append(embedding.detach().cpu().numpy()) # because we are ignoring the CLS and SEP tokens.
      #inputs_embedding.append(embedding.detach().cpu().numpy()[shift_left:shift_right])
      
      #del ids, embedding
      #gc.collect()
      #torch.cuda.empty_cache()


  return inputs_embedding

In [5]:
path_meltome = 'datasets/FLIP/splits/meltome/splits/mixed_split.csv'

splits_meltome = pd.read_csv(path_meltome, sep=',')
train, val, test = train_test_validation_splits(splits_meltome)
print('done')

done


In [8]:
splits_meltome[splits_meltome['sequence'].str.len()< 50].sequence.str.len()

66       44
794      44
1086     42
1487     44
1546     44
1658     40
1665     43
1681     47
1817     41
1911     28
2557     27
2591     47
2866     35
3286     47
3287     46
5331     41
9148     39
9248     46
9755     20
10900    45
14037    41
16365    36
16366    36
19879    32
24037    39
27089    44
Name: sequence, dtype: int64

In [5]:
!pip install accelerate bitsandbytes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
#from accelerate import init_empty_weights
#from accelerate import infer_auto_device_map
max_memory_mapping = {0: "10GB", 'cpu': "5GB"}

tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False)
model = BertModel.from_pretrained("Rostlab/prot_bert", low_cpu_mem_usage=True, max_memory=max_memory_mapping)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model = model.to(device=device)
model = model.eval()
if torch.cuda.is_available():
  model = model.half()

Some weights of the model checkpoint at Rostlab/prot_bert were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
def pooling_and_final_representation(path):
  # Due to npz compression on each embedded sequence
  sample = np.load(path)
  embedded_seq = torch.from_numpy( sample[sample.files[0]] ).to(torch.float32)
  embedded_seq = embedded_seq.view(embedded_seq.size(0), -1).permute(1,0)
  GlobalAvgpooling = torch.nn.AvgPool1d(kernel_size = embedded_seq.size(1) , stride = 1, padding = 0)
  pooled_seq = GlobalAvgpooling(embedded_seq)

  return pooled_seq.permute(1,0)

#the test
import pdb
def pooled_set_of_sequences(folder_path):
  list_embeddings = []
  iter_paths = os.listdir(folder_path)
  for path in tqdm(iter_paths):
    list_embeddings.append( pooling_and_final_representation(folder_path + path) )

  #import pdb;pdb.set_trace()
  representation = torch.stack(list_embeddings)
  representation = representation.view(representation.size(0),-1)
  return representation

#train_embeddings = pooled_set_of_sequences(train_tmp_folder)
#test_embeddings = pooled_set_of_sequences(test_tmp_folder)
#val_embeddings = pooled_set_of_sequences(val_tmp_folder)

  0%|          | 0/22335 [00:00<?, ?it/s]

  0%|          | 0/3134 [00:00<?, ?it/s]

  0%|          | 0/2482 [00:00<?, ?it/s]

In [7]:
dir_pre_generated_embeddings = ''
path_train= dir_pre_generated_embeddings+'train_set_embeddings.pth'
path_test= dir_pre_generated_embeddings+'test_set_embeddings.pth'
path_val= dir_pre_generated_embeddings+'val_set_embeddings.pth'


if os.path.isfile(path_train) and os.path.isfile(path_test) and os.path.isfile(path_val):
    #if path.exists(train_tmp_folder) == False or path.exists(test_tmp_folder) == False or path.exists(val_tmp_folder) == False:
    print ("Creating Embeddings...")
    embed_dataset(train.sequence, model, tokenizer, device, train_tmp_folder)
    embed_dataset(test.sequence, model, tokenizer, device, test_tmp_folder)
    embed_dataset(val.sequence, model, tokenizer, device, val_tmp_folder)
    print ("DONE!!...")


print ("Loading Premade Embeddings...")

train_embeddings = pooled_set_of_sequences(train_tmp_folder)
torch.save(train_embeddings, 'train_set_embeddings.pth')
test_embeddings = pooled_set_of_sequences(test_tmp_folder)
torch.save(test_embeddings, 'test_set_embeddings.pth')
val_embeddings = pooled_set_of_sequences(val_tmp_folder)
torch.save(val_embeddings, 'val_set_embeddings.pth')

print ("Loaded")

  0%|          | 0/22335 [00:00<?, ?it/s]

  0%|          | 0/3134 [00:00<?, ?it/s]

  0%|          | 0/2482 [00:00<?, ?it/s]

In [9]:
!ls -1 /content/tmp

test_embeddinds
train_embeddinds
val_embeddinds


In [29]:
import torch.utils.data as data_utils

batch_size=50


trainDataset = data_utils.TensorDataset(train_embeddings, torch.from_numpy( train.target.to_numpy() ))
testDataset = data_utils.TensorDataset(test_embeddings, torch.from_numpy( test.target.to_numpy() ))
valDataset = data_utils.TensorDataset(val_embeddings, torch.from_numpy( val.target.to_numpy() ))

train_loader = data_utils.DataLoader(trainDataset, batch_size=batch_size, shuffle=True)
test_loader = data_utils.DataLoader(testDataset, batch_size=batch_size, shuffle=True)
val_loader = data_utils.DataLoader(valDataset, batch_size=batch_size, shuffle=True)

In [32]:
class Flatten(torch.nn.Module):
    def __init__(self):
        super(Flatten, self).__init__()
    
    def forward(self, input):
        return input.view(input.size(0), -1)   

class regressionHead(torch.nn.Module):

    def __init__(self, kernel):
        super(regressionHead, self).__init__()
        #self.input_shape = input_shape # pos0 = #channels, pos1 = #diagonal comps, or viseversa


        self.FFNN = torch.nn.Sequential(
            torch.nn.Linear(1024, 512),
            torch.nn.ReLU(), #nn.ReLU(),
            torch.nn.Linear(512, 512),
            torch.nn.ReLU(), #nn.ReLU(),
            torch.nn.Linear(512, 1)
        )
        
        #self.encoder.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.kaiming_normal_(module.weight)

    def forward(self, x):
        #x = self.pooling(x).mean(dim=0)
        z = self.FFNN(x)
        return z #z.permute(0,2,1) 

In [33]:
def train_LLMRegresor(train_iterator, val_iterator, model, device, criterion, optimizer, epoch_num):
    
        
        val_loss = []
        
        model.to(device)

        for epoch in range(epoch_num):
            model.train() 
            for i, (input, label) in enumerate(train_iterator):
                optimizer.zero_grad()
                
                input = input.to(dtype=torch.float32, device=device)
                label = label.to(dtype=torch.float32, device=device) 
                out = model(input)
                loss = criterion(out, label.unsqueeze(-1))
                loss.backward() 
                optimizer.step()

            with torch.no_grad(): # evaluate validation loss here 

                model.eval()
                val_loss_epochs = []

                for (inputval, labelval) in val_iterator:
                    
                    inputval = inputval.to(device)
                    labelval = labelval.to(device)
                    outval = model(inputval)
                    lossval = criterion(outval, labelval.unsqueeze(-1)) # Calculate validation loss 
                    val_loss_epochs.append(lossval.item())

                val_loss_epoch = np.mean(val_loss_epochs)
                val_loss.append(round(val_loss_epoch, 3))

            print('epoch: %d loss: %.3f val loss: %.3f' % (epoch + 1, loss.item(), val_loss_epoch))

            '''# evalutate whether validation loss is dropping; if not, stop
            if epoch > 21:
                if val_loss[-1] >= np.min(val_loss[:-20]): 
                    print('Finished training at epoch {0}'.format(epoch))
                    return epoch'''

In [ ]:
model = regressionHead(kernel=3)
opt = torch.optim.Adam(model.parameters(), lr=1e-4)
loss= torch.nn.MSELoss()
epochs = 3500
train_LLMRegresor(train_loader, val_loader, model, device, loss, opt, epochs)

epoch: 1 loss: 120.748 val loss: 113.592
epoch: 2 loss: 152.514 val loss: 112.613
epoch: 3 loss: 59.104 val loss: 112.650
epoch: 4 loss: 95.823 val loss: 113.119
epoch: 5 loss: 143.580 val loss: 113.831
epoch: 6 loss: 51.005 val loss: 113.953
epoch: 7 loss: 40.938 val loss: 112.403
epoch: 8 loss: 130.843 val loss: 113.136
epoch: 9 loss: 94.524 val loss: 112.687
epoch: 10 loss: 71.859 val loss: 113.492
epoch: 11 loss: 93.797 val loss: 112.514
epoch: 12 loss: 105.875 val loss: 113.152
epoch: 13 loss: 120.767 val loss: 112.483
epoch: 14 loss: 131.491 val loss: 112.563
epoch: 15 loss: 188.326 val loss: 116.746
epoch: 16 loss: 54.448 val loss: 112.688
epoch: 17 loss: 98.666 val loss: 112.956
epoch: 18 loss: 180.381 val loss: 113.722
epoch: 19 loss: 207.329 val loss: 112.615
epoch: 20 loss: 175.025 val loss: 113.337
epoch: 21 loss: 160.468 val loss: 112.963
epoch: 22 loss: 147.812 val loss: 113.447
epoch: 23 loss: 157.584 val loss: 112.895
epoch: 24 loss: 100.469 val loss: 113.227
epoch: 25 

In [ ]:
test_set, test_labels = test_loader.dataset.tensors

In [ ]:
import math
test_set = test_set.to(dtype=torch.float32, device=device)
test_labels = test_labels.to(dtype=torch.float32, device=device) 
outcome = model(test_set)
loss_test = loss(outcome, test_labels.unsqueeze(-1))
print('MSE: ' + str(loss_test))
print('RMSE: ' + str(torch.sqrt(loss_test)))

In [ ]:



'''sequence_Example = "A E T C Z A O"
sequence_Example = re.sub(r"[UZOB]", "X", sequence_Example)
encoded_input = tokenizer(sequence_Example, return_tensors='pt')
output = model(**encoded_input)
print(output)'''